<a href="https://colab.research.google.com/github/samibahig/IFT6390/blob/main/CNN_DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.7.0+cu101
Torchvision Version:  0.8.1+cu101


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
! unzip /content/QuickDrawPyTorchFiles.zip  -d /content

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: /content/data/QuickDrawFiles/QuickDraw_tv/test/QuickDrawTest/test17747.jpg  
  inflating: /content/data/QuickDrawFiles/QuickDraw_tv/test/QuickDrawTest/test9643.jpg  
  inflating: /content/data/QuickDrawFiles/QuickDraw_tv/test/QuickDrawTest/test57705.jpg  
  inflating: /content/data/QuickDrawFiles/QuickDraw_tv/test/QuickDrawTest/test38623.jpg  
  inflating: /content/data/QuickDrawFiles/QuickDraw_tv/test/QuickDrawTest/test59204.jpg  
  inflating: /content/data/QuickDrawFiles/QuickDraw_tv/test/QuickDrawTest/test52239.jpg  
  inflating: /content/data/QuickDrawFiles/QuickDraw_tv/test/QuickDrawTest/test41994.jpg  
  inflating: /content/data/QuickDrawFiles/QuickDraw_tv/test/QuickDrawTest/test23354.jpg  
  inflating: /content/data/QuickDrawFiles/QuickDraw_tv/test/QuickDrawTest/test24541.jpg  
  inflating: /content/data/QuickDrawFiles/QuickDraw_tv/test/QuickDrawTest/test8675.jpg  
  inflating: /content/da

In [ ]:
! ls -l /content/data/QuickDrawFiles/QuickDraw_tv/val
! ls -l /content/data/QuickDrawFiles/QuickDraw_tv/train
! ls -l /content/data/QuickDrawFiles/QuickDraw_tv/test

total 24
drwx------ 2 root root 4096 Dec 12 06:09 0
drwx------ 2 root root 4096 Dec 12 06:09 1
drwx------ 2 root root 4096 Dec 12 06:09 2
drwx------ 2 root root 4096 Dec 12 06:09 3
drwx------ 2 root root 4096 Dec 12 06:09 4
drwx------ 2 root root 4096 Dec 12 06:09 5
total 72
drwx------ 2 root root 12288 Dec 12 06:09 0
drwx------ 2 root root 12288 Dec 12 06:09 1
drwx------ 2 root root 12288 Dec 12 06:09 2
drwx------ 2 root root 12288 Dec 12 06:09 3
drwx------ 2 root root 12288 Dec 12 06:09 4
drwx------ 2 root root 12288 Dec 12 06:09 5
total 1992
drwxr-xr-x 2 root root 2039808 Nov 17 13:13 QuickDrawTest


In [ ]:
! ls -l /content/data

total 4
drwxr-xr-x 3 root root 4096 Dec 17 21:15 QuickDrawFiles


In [ ]:
# MN:  Code to copy files from Validation to Training set
!cp /content/data/QuickDrawFiles/QuickDraw_tv/val/0/*.* /content/data/QuickDrawFiles/QuickDraw_tv/train/0/.
!cp /content/data/QuickDrawFiles/QuickDraw_tv/val/1/*.* /content/data/QuickDrawFiles/QuickDraw_tv/train/1/.
!cp /content/data/QuickDrawFiles/QuickDraw_tv/val/2/*.* /content/data/QuickDrawFiles/QuickDraw_tv/train/2/.
!cp /content/data/QuickDrawFiles/QuickDraw_tv/val/3/*.* /content/data/QuickDrawFiles/QuickDraw_tv/train/3/.
!cp /content/data/QuickDrawFiles/QuickDraw_tv/val/4/*.* /content/data/QuickDrawFiles/QuickDraw_tv/train/4/.
!cp /content/data/QuickDrawFiles/QuickDraw_tv/val/5/*.* /content/data/QuickDrawFiles/QuickDraw_tv/train/5/.

In [ ]:
! ls -l /content/data/QuickDrawFiles/QuickDraw_tv/train/4/. | wc -l

251


In [ ]:
!ls -l ./data/QuickDrawFiles/QuickDraw_tv/test/QuickDrawTest | wc -l

60001


In [ ]:
# Top level data directory. Here we assume the format of the directory conforms 
#   to the ImageFolder structure
#data_dir = "./data/hymenoptera_data"
data_dir="./data/QuickDrawFiles/QuickDraw_tv"  #QuickDraw data for torchvision

#experimenting test dir 
test_dir = "./data/QuickDrawFiles/QuickDraw_tv/test/QuickDrawTest"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
#model_name = "squeezenet" #(72 pc on val set after 50 epoch)
#model_name = "inception" #(86 pc with full training on 20 epochs)
model_name="densenet" #(74 pc and 89 pc with full training)  (87 pc on kaggle, try again with full trainset)
##model_name="resnet" # (86 pc with full training on 20 epochs)
##model_name = "vgg" #(87 pc with full training on 20 epochs and same with 50 epochs)

# Number of classes in the dataset
num_classes = 6

# Batch size for training (change depending on how much memory you have)
batch_size = 32

# Batch size for test should be 1 (so we get correct filenames)
test_batch_size = 1

# Number of epochs to train for 
num_epochs = 90

# Flag for feature extracting. When False, we finetune the whole model, 
#   when True we only update the reshaped layer params
feature_extract = False

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    elif model_name == "inception":
        """ Inception v3 
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size

#The model is initialized in the call below.
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth



DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
#transforms.RandomVerticalFlip(),  ##ADDED THIS BUT REMOVED AFTER ACC DROPPED
#transforms.CenterCrop(input_size),
#transforms.GaussianBlur(kernel_size, sigma=(0.1, 2.0)), #Added
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),  
        transforms.RandomRotation(40),  #TRT WITH 20
        #transforms.Pad(10), ##ADDED
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2), ##ADDED
        transforms.ToTensor(),
        transforms.RandomErasing(p=0.75,scale=(0.02, 0.1),value=1.0, inplace=False),  ##ADDED
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
     'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val', 'test']}

#TODO:  Can u create different dataloaders and put them in this dictionary as "train" and "val" dataloaders?
#comment out below code & create my_dataloader_dict with 2 data_loaders for train and val & check if it works.

# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val', 'test']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

##TODO
# Before running the test loop, create your own test dataloader with batch_size=1, same trasforms as done here, add that data_loader to the dictionary and then run the loop

Initializing Datasets and Dataloaders...


In [ ]:
print (dataloaders_dict)

{'train': <torch.utils.data.dataloader.DataLoader object at 0x7febd038c668>, 'val': <torch.utils.data.dataloader.DataLoader object at 0x7febd038c4e0>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7febd038c438>}


In [ ]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are 
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
optimizer_ft = optim.SGD(params_to_update, lr=0.00065, momentum=0.9)

Params to learn:
	 features.conv0.weight
	 features.norm0.weight
	 features.norm0.bias
	 features.denseblock1.denselayer1.norm1.weight
	 features.denseblock1.denselayer1.norm1.bias
	 features.denseblock1.denselayer1.conv1.weight
	 features.denseblock1.denselayer1.norm2.weight
	 features.denseblock1.denselayer1.norm2.bias
	 features.denseblock1.denselayer1.conv2.weight
	 features.denseblock1.denselayer2.norm1.weight
	 features.denseblock1.denselayer2.norm1.bias
	 features.denseblock1.denselayer2.conv1.weight
	 features.denseblock1.denselayer2.norm2.weight
	 features.denseblock1.denselayer2.norm2.bias
	 features.denseblock1.denselayer2.conv2.weight
	 features.denseblock1.denselayer3.norm1.weight
	 features.denseblock1.denselayer3.norm1.bias
	 features.denseblock1.denselayer3.conv1.weight
	 features.denseblock1.denselayer3.norm2.weight
	 features.denseblock1.denselayer3.norm2.bias
	 features.denseblock1.denselayer3.conv2.weight
	 features.denseblock1.denselayer4.norm1.weight
	 features.de

In [ ]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/89
----------


KeyboardInterrupt: ignored

In [ ]:
class MyEnsemble(nn.Module): 
    def __init__(self, modelA, modelB, nb_classes=6): 
        super(MyEnsemble, self).__init__() 
        self.modelA = modelA 
        self.modelB = modelB 
        # Remove last linear layer 
        self.modelA.fc = nn.Identity() self.modelB.fc = nn.Identity() 
        # Create new classifier 
        self.classifier = nn.Linear(2048+512, nb_classes) def forward(self, x): x1 = self.modelA(x.clone()) 
        # clone to make sure x is not changed by inplace methods 
        x1 = x1.view(x1.size(0), -1) x2 = self.modelB(x) x2 = x2.view(x2.size(0), -1) x = torch.cat((x1, x2), dim=1) x = self.classifier(F.relu(x)) return x 
        # Train your separate models # ... 
        # We use pretrained torchvision models here 
        modelA = models.resnet50(pretrained=True) modelB = models.resnet18(pretrained=True) 
        # Freeze these models for param in modelA.parameters(): 
        param.requires_grad_(False) for param in modelB.parameters(): param.requires_grad_(False) 
        # Create ensemble model 
        model = MyEnsemble(modelA, modelB) 
        x = torch.randn(1, 3, 224, 224) output = model(x)